In [1]:
from pathlib import Path
from mri_data import utils
import nibabel as nib
import os
import json
import re
import statistics
import numpy as np
import pyperclip
import subprocess
import pandas as pd

from mri_data import file_manager as fm

In [4]:
drive_root = fm.get_drive_root()
dataroot = drive_root / "3Tpioneer_bids"
# training_work_home = Path("/home/srs-9/Projects/prl_project/training_work_dirs")
training_work_home = drive_root / "srs-9/prl_project/training_work_dirs"
work_dir = training_work_home / "test_train0_swinunet" #! edit this
ensemble_dir = work_dir / "ensemble_output"
datalist_file_src = "/home/srs-9/Projects/prl_project/training/test_train0/datalist.json"
datalist_file = work_dir / "datalist.json"

prl_info = pd.read_csv("/home/srs-9/Projects/prl_project/data/PRL_labels.csv")

def subid_from_subject(subject):
    return int(re.search(r"(\d{4})", subject)[1])
prl_info['subid'] = prl_info['ID'].map(subid_from_subject)
prl_info.set_index("subid", inplace=True)

In [9]:
with open(datalist_file, 'r') as f:
    datalist = json.load(f)

subject_paths = {}
for item in datalist['training']:
    label = str(drive_root / Path(item['label']).relative_to("/media/smbshare"))
    assert os.path.exists(label)
    subject = re.search(r"(sub-ms\d{4})", label)[1]
    subid = re.search(r"(\d{4})", subject)[1]
    subject_paths[subid] = {}
    subject_paths[subid]['label'] = label
    session = re.search(r"(ses-\d{8})", label)[1]
    subject_paths[subid]['image_dir'] = dataroot / subject / session

In [16]:
subid = 1010
image_dir = subject_paths[str(subid)]['image_dir']
images = ["flair.nii.gz", "phase.nii.gz", "t1.nii.gz"]
image_paths = [image_dir / im for im in images]

label_paths = [subject_paths[str(subid)]['label']]

cmd = utils.open_itksnap_workspace_cmd(image_paths, label_paths, win=True)
print(cmd)
print("TOTAL PRL:", prl_info.loc[subid, 'Total PRL'])
pyperclip.copy(cmd)
# subprocess.Popen(cmd.split(" "))

itksnap -g H:/3Tpioneer_bids/sub-ms1010/ses-20180208/flair.nii.gz -o H:/3Tpioneer_bids/sub-ms1010/ses-20180208/phase.nii.gz H:/3Tpioneer_bids/sub-ms1010/ses-20180208/t1.nii.gz -s H:/3Tpioneer_bids/sub-ms1010/ses-20180208/lesion.t3m20/lesion_and_prl_labels_tmp.nii.gz
TOTAL PRL: 3
